___


# Energy Informatics 2 Excercise 2025
## Exercise 9 & 10 - Dataanalysis & Forecasting</span>

___



# **1. Introduction**


Welcome to our exercise notebook on data analysis and time series forecasting with sktime! This notebook will explore some tools for analyzing data and further show a small pipline approach to forecast future values. This notebook will provide you with some insights and hands-on experience in working with time series data. Therefore, let's get started and dive into the exciting world of data analysis and time series forecasting!

This is an interactive notebook and also includes some work assignments. Typical tasks are adding lines of code, documenting observations. Work orders are always marked in the color <span style="color:#A00000"> **red** </span>. We suggest you work in pairs or small groups so that you can share observations and discuss the tasks together.

___


## Agenda

1. Introduction, Agenda, learning goals and data loading

2. Data Analysis

3. Forecasting Excercise

___


##  Learning Goals

### Learning goals data analysis and time series analysis

    - Enumerate different properties of time series e.g. seasonality    
    - Analyze energy time series and recognize and evaluate seasonal patterns
    - Derive and design appropriate calendrical features by considering seasonality and autocorellation function.


### Learning goals machine learning
    - Know basic Concepts of skTime
    - Know the basic Concept of pipelining

___



## IEEE Case 9



___


# **1. Data loading and first preprocessing**
As a basis for the data analysis we need data in the first place. This publically available data set is described here: [Full Dataset Introduction](https://data.open-power-system-data.org/household_data/2020-04-15)

Here we have taken only a subset of the data, since we want to deal with only one building. Our choice is the industrial building 3. All the data is scaled in kWh. The building has an installed pv and an energy demand.



In [1]:
import pandas as pd
data = pd.read_csv('https://data.open-power-system-data.org/household_data/2020-04-15/household_data_60min_singleindex.csv', date_format='%Y-%m-%dT%H:%M:%SZ', index_col = "utc_timestamp", parse_dates=True , sep=',')
data.index = pd.to_datetime(data.index, utc=True)

In [2]:
import statsmodels.api as sm
from matplotlib import pylab
from pylab import *


# Basic configuration to get beautiful pictures
pylab.rcParams['figure.figsize'] = (16, 9)

# Get the relevant data for this excercise and resample it to hourly resolution to save runtime complexity
data["demand"] = data["DE_KN_industrial3_grid_import"].diff(1)
data["solar"] = (data["DE_KN_industrial3_pv_facade"].diff(1) + data["DE_KN_industrial3_pv_roof"].diff(1))

# shift the index of the data by one interval to get the correct time alignment
data.index = data.index - pd.Timedelta(hours=1)

# Omit data without values
data = data[["demand","solar"]].dropna()
# Let the data start with a full day and end with a full day
data =  data[(data.index >= pd.to_datetime("2016-11-03",utc=True)) &  (data.index < pd.to_datetime("2017-06-04",utc=True))]
data = data.asfreq('1h')

# **2. Data Analysis**

<img src="https://imgs.xkcd.com/comics/data_trap.png" width="400" height="400">

[This xkcd comic you can find here](https://xkcd.com/2582/)